## Automatic Content Tagging with Gorilla LLM

More information about Gorilla can be found here: https://github.com/ShishirPatil/gorilla.

In [1]:
# Import Chat completion template and set-up variables
!pip install openai
import openai
import urllib.parse

openai.api_key = "EMPTY" # Key is ignored and does not matter
openai.api_base = "http://zanino.millennium.berkeley.edu:8000/v1"
# Alternate mirrors
# openai.api_base = "http://34.132.127.197:8000/v1"

# Report issues
def raise_issue(e, model, prompt):
    issue_title = urllib.parse.quote("[bug] Hosted Gorilla: <Issue>")
    issue_body = urllib.parse.quote(f"Exception: {e}\nFailed model: {model}, for prompt: {prompt}")
    issue_url = f"https://github.com/ShishirPatil/gorilla/issues/new?assignees=&labels=hosted-gorilla&projects=&template=hosted-gorilla-.md&title={issue_title}&body={issue_body}"
    print(f"An exception has occurred: {e} \nPlease raise an issue here: {issue_url}")

# Query Gorilla server
def get_gorilla_response(prompt="I would like to translate from English to French.", model="gorilla-7b-hf-v1"):
  try:
    completion = openai.ChatCompletion.create(
      model=model,
      messages=[{"role": "user", "content": prompt}]
    )
    return completion.choices[0].message.content
  except Exception as e:
    raise_issue(e, model, prompt)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.1 MB/s eta 0:00:00


In [5]:
prompt = "I would like to do automatic content tagging. As input, I have a question text and a list of possible tags. The texts are in portuguese."
print(get_gorilla_response(prompt, model="gorilla-7b-hf-v1"))

<<<domain>>>: Natural Language Processing Zero-Shot Classification
<<<api_call>>>: classifier = pipeline('zero-shot-classification', model='vicgalle/xlm-roberta-large-xnli-anli')
<<<api_provider>>>: Hugging Face Transformers
<<<explanation>>>: 1. Import the pipeline function from the transformers library.
2. Create a zero-shot classification model using the pipeline function, loading the 'vicgalle/xlm-roberta-large-xnli-anli' model.
3. This model will classify text into predefined categories based on semantic similarity, even if it has not been explicitly trained on those categories.
4. Use the classifier to classify the input question text into one of the given tags.
<<<code>>>:
from transformers import pipeline

def load_model(model_name):
    classifier = pipeline('zero-shot-classification', model=model_name)
    return classifier

def process_data(classifier, text, tags):
    response = classifier(text, tags)
    return response

question = "Qual a principal diferença entre fontes 

In [6]:
!pip install --quiet transformers sentencepiece

In [21]:
import pandas as pd
import numpy as np

tags_df = pd.read_csv("ENEM_Reference_Matrix.csv")
questions_df = pd.read_csv("ENEM_tagged_questions.csv")

tags_df.head()

,Discipline,Skill_Number,Skill_Description
0,"Linguagens, Códigos e suas Tecnologias",1,Identificar as diferentes linguagens e seus re...
1,"Linguagens, Códigos e suas Tecnologias",2,Recorrer aos conhecimentos sobre as linguagens...
2,"Linguagens, Códigos e suas Tecnologias",3,Relacionar informações geradas nos sistemas de...
3,"Linguagens, Códigos e suas Tecnologias",4,Reconhecer posições críticas aos usos sociais ...
4,"Linguagens, Códigos e suas Tecnologias",5,Associar vocábulos e expressões de um texto em...


In [8]:
questions_df.head()

,Year,SG_AREA,TX_COR,CO_PROVA,Question Number,Question Text,Ability Code
0,2022.0,MT,AZUL,1075,Questão 136 - Matemática e suas Tecnologias,Uma máquina em operação tem sua temperat...,25
1,2022.0,MT,AZUL,1075,Questão 137 - Matemática e suas Tecnologias,A World Series é a decisão do campeonat...,28
2,NaN,MT,AZUL,1075,Questão 138 - Matemática e suas Tecnologias,O gráfico apresenta os totais de receitas e de...,27
3,NaN,MT,AZUL,1075,Questão 139 - Matemática e suas Tecnologias,Um casal está reformando a cozinha de c...,11
4,NaN,MT,AZUL,1075,Questão 140 - Matemática e suas Tecnologias,Foram convidadas 32 equipes para um torn...,3


In [17]:
from transformers import pipeline

def load_model(model_name):
    classifier = pipeline('zero-shot-classification', model=model_name)
    return classifier

def process_data(classifier, text, tags):
    response = classifier(text, tags)
    return response

question = questions_df.iloc[0]["Question Text"]
tags = tags_df["Skill_Description"].values

# Load the model
classifier = load_model('vicgalle/xlm-roberta-large-xnli-anli')

# Process the data
response = process_data(classifier, question, tags)
print(response)

{'sequence': 'Uma  máquina  em  operação  tem  sua  temperatura  T  monitorada  por  meio  de  um\nregistro gráfico, ao longo do tempo t. Essa máquina possui um pistão cuja velocidade V\n\nvaria com a temperatura T da máquina, de acordo com a expressão V = T  –  4.  Após  a\nmáquina  funcionar  durante  o  intervalo  de  tempo  de  10  horas,  o  seu  operador  analisa  o\nregistro gráfico, apresentado na figura, para avaliar a necessidade de eventuais ajustes,\n\n2 \n\nsabendo  que  a  máquina  apresenta  falhas  de  funcionamento  quando  a  velocidade  do\npistão se anula.\n\nt (h)\n\n10\n\nQuantas vezes a velocidade do pistão se anulou durante as 10 horas de funcionamento?\n\nT (ºC)\n\n4\n\n2\n\n0\n\n-2\n\nA\n\nB\n\nC\n\nD\n\nE\n\n1\n\n2\n\n3\n\n4\n\n5\n\n/', 'labels': ['Utilizar informações expressas em gráficos ou tabelas para fazer inferências.', 'Identificar relações entre grandezas e unidades de medida.', 'Resolver situação-problema envolvendo conhecimentos numéricos.', 'Ident